## Ames Kaggle Dataset

Initial inspection of the data and EDA to choose the best variables for the analysis

In [ ]:
#Libraries we will be using

import pandas as pd

In [13]:
#import and read the train data

train_enriched = pd.read_csv('train_enriched.csv')

train_enriched.shape

(1460, 94)